# Analysis of presence of variants by impact in lineages

## Setup

In [43]:
import os
os.getcwd()
os.chdir('/FastData/czirion/ServerDatabase/scripts')
import query_database as qdb
import pandas as pd
import duckdb
os.getcwd()

'/mnt/FastData/czirion/ServerDatabase/scripts'

In [44]:
mydb = '/FastData/czirion/Crypto_Desjardins/results_2024-10-22/02.Dataset/database.db'
# mydb = '/FastData/czirion/DiversityPipeline/results_joined_R1R2/02.Dataset/database.db'

## Select parameters for the analysis

In [45]:
impact = "HIGH"
lineages = ("VNI", "VNII", "VNBI", "VNBII")
lin_names = "_".join(lineages)
phylogenetic_category = ("monophyletic", "paraphyletic", "unique")
phylogenetic_category_name = "no-poly"
below_percent_strains = 100
remove_mutated_genes = True
remove_mutated_genes_name = "no-mut"
min_mapq = 20
min_depth = 0.5
max_depth = 1.5
max_repeat = 0.2

## Presence of variants in transcripts and strains
Get the variants with the selected impact and the strains and transcripts where they are present.

In [46]:
con = duckdb.connect(database=mydb, read_only=False)
query = f"""SELECT presence.lineage, presence.var_id, 
                samples.strain, effects.transcript_id
            FROM presence
            JOIN effects ON presence.var_id = effects.var_id
            JOIN samples ON presence.sample = samples.sample 
            WHERE effects.impact = '{impact}'
            """
vars_presence = con.execute(query).fetch_df()
con.close()

Remove variants whose effect does not affect a single transcript (e.g. fused genes)

In [47]:
vars_presence = vars_presence[vars_presence['transcript_id'].notna()]
vars_presence

,lineage,var_id,strain,transcript_id
0,VNBII,var_VNBII_504,Bt81,CNAG_00853-mR1
1,VNBII,var_VNBII_4199,Bt81,CNAG_00782-mR1
2,VNBII,var_VNBII_4634,Bt81,CNAG_00777-mR1
3,VNBII,var_VNBII_5130,Bt81,CNAG_00768-mR1
4,VNBII,var_VNBII_5132,Bt81,CNAG_00768-mR1
...,...,...,...,...
168797,VNBII,var_VNBII_121248,Bt39,CNAG_06812-mR1
168798,VNBII,var_VNBII_121305,Bt39,CNAG_06812-mR1
168799,VNBII,var_VNBII_114133,Bt39,CNAG_05304-mR2
168800,VNBII,var_VNBII_115930,Bt39,CNAG_05325-mR1


## Get variants in selected phylogenetic categories

Load the phylogenetic category of each variant

In [48]:
var_phylogeny = pd.read_csv("/FastData/czirion/Crypto_Desjardins/fungal_pop/data/variant_status.csv", header = 0)

Add the phylogenetic category to the variants table

In [49]:
vars_presence_phylo = vars_presence.merge(var_phylogeny, on = ["var_id", "strain"], how = "left")

Remove variants that are not in the selected phylogenetic categories

In [50]:
vars_presence_phylo_filtered = vars_presence_phylo[vars_presence_phylo["category"].isin(phylogenetic_category)]

In [51]:
# vars_presence_phylo_grouped = vars_presence_phylo_filtered.groupby(['var_id']).count()
# vars_presence_phylo_grouped
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.hist(vars_presence_phylo_grouped['strain'], bins= 100)
# plt.xlim(0, 30)
# plt.show()

Total, filltered out, remaining variants

In [52]:
vars_presence['var_id'].nunique(), vars_presence['var_id'].nunique() - vars_presence_phylo_filtered['var_id'].nunique(), vars_presence_phylo_filtered['var_id'].nunique()

(11917, 5033, 6884)

In [53]:
vars_presence_phylo_filtered

,lineage,var_id,strain,transcript_id,category
0,VNBII,var_VNBII_504,Bt81,CNAG_00853-mR1,monophyletic
1,VNBII,var_VNBII_4199,Bt81,CNAG_00782-mR1,monophyletic
2,VNBII,var_VNBII_4634,Bt81,CNAG_00777-mR1,monophyletic
6,VNBII,var_VNBII_6613,Bt81,CNAG_00745-mR1,monophyletic
7,VNBII,var_VNBII_7802,Bt81,CNAG_00726-mR1,monophyletic
...,...,...,...,...,...
168681,VNBII,var_VNBII_115804,Bt39,CNAG_05321-mR2,paraphyletic
168683,VNBII,var_VNBII_115974,Bt39,CNAG_05326-mR1,monophyletic
168685,VNBII,var_VNBII_118252,Bt39,CNAG_06849-mR1,monophyletic
168701,VNBII,var_VNBII_113216,Bt39,CNAG_05298-mR1,monophyletic


## Get variants in selected percent of strains

Number of strains per lineage

In [54]:
con = duckdb.connect(database=mydb, read_only=False)
query = "SELECT lineage,strain FROM samples"
result = con.execute(query).fetch_df()
con.close()
strains_per_lineage = result.groupby("lineage")['strain'].nunique().reset_index(name='total_strains')
strains_per_lineage

,lineage,total_strains
0,VNBI,122
1,VNBII,64
2,VNI,185
3,VNII,16


Number and percentage of strains with each variant

In [55]:
vars_percent = vars_presence.groupby(["lineage","var_id", "transcript_id"])['strain'].nunique().reset_index(name='num_strains')
vars_percent = vars_percent.merge(strains_per_lineage, on='lineage', how='left')
vars_percent['percent_strains'] = (vars_percent['num_strains'] / vars_percent['total_strains']) * 100
vars_percent['percent_strains'] = vars_percent['percent_strains'].round(2)
vars_percent = vars_percent.drop(columns=['total_strains'])

Remove variants present in and above the selected percentage of strains.

In [56]:
vars_percent_filtered = vars_percent[vars_percent['percent_strains'] < below_percent_strains]

Total, filtered out, remaining variants

In [57]:
vars_percent['var_id'].nunique(), vars_percent['var_id'].nunique() - vars_percent_filtered['var_id'].nunique(), vars_percent_filtered['var_id'].nunique()

(11917, 170, 11747)

In [58]:
vars_percent_filtered

,lineage,var_id,transcript_id,num_strains,percent_strains
0,VNBI,var_VNBI_100004,CNAG_01914-mR1,1,0.82
1,VNBI,var_VNBI_101030,CNAG_01901-mR1,31,25.41
2,VNBI,var_VNBI_101058,CNAG_01901-mR1,52,42.62
3,VNBI,var_VNBI_101109,CNAG_01900-mR1,1,0.82
4,VNBI,var_VNBI_101277,CNAG_01897-mR1,22,18.03
...,...,...,...,...,...
13792,VNII,var_VNII_89815,CNAG_07042-mR1,11,68.75
13793,VNII,var_VNII_9022,CNAG_00698-mR1,1,6.25
13794,VNII,var_VNII_908,CNAG_00080-mR1,12,75.00
13795,VNII,var_VNII_914,CNAG_00080-mR1,12,75.00


## Effects and annotations
Get the table of the effects, join it with the annotation of the reference genomes.

In [59]:
effects = qdb.effects(db = mydb, impact = (impact,), lineage = lineages)
keep_columns = ['lineage', 'strain', 'var_id', 'transcript_id', 'gene_id', 'impact', 'effect_type', 'mean_mapq', 'mean_depth_normalized']
effects = effects[keep_columns]


        SELECT samples.dataset, samples.strain, presence.sample, presence.lineage,
            variants.var_id, chromosome_names.chromosome,
            variants.pos AS position, variants.ref AS reference, variants.alt AS alternative,
            effects.gene_name, effects.gene_id, effects.transcript_id,
            effects.impact, effects.effect_type, effects.effect,
            effects.codon_change, effects.amino_acid_change, effects.amino_acid_length,
            effects.transcript_biotype, effects.gene_coding, effects.exon_rank,
            mapq_depth.mean_depth_normalized, mapq_depth.mean_mapq
        FROM variants 
        JOIN chromosome_names ON variants.accession = chromosome_names.accession
        JOIN presence ON variants.var_id = presence.var_id
        JOIN effects ON variants.var_id = effects.var_id
        JOIN samples ON presence.sample = samples.sample
        LEFT JOIN mapq_depth ON mapq_depth.feature_id = effects.transcript_id AND mapq_depth.sample = presence.sampl

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [60]:
effects

,lineage,strain,var_id,transcript_id,gene_id,impact,effect_type,mean_mapq,mean_depth_normalized
0,VNBI,NRHc5022.ENR,var_VNBI_312896,CNAG_04268-mR1,CNAG_04268,HIGH,SPLICE_SITE_DONOR+INTRON,60.0,0.78
1,VNBI,NRHc5022.ENR,var_VNBI_312992,CNAG_04272-mR1,CNAG_04272,HIGH,SPLICE_SITE_DONOR+INTRON,60.0,0.86
2,VNBI,NRHc5022.ENR,var_VNBI_314791,CNAG_04314-mR2,CNAG_04314,HIGH,START_LOST,60.0,0.80
3,VNBI,NRHc5022.ENR,var_VNBI_315667,CNAG_04331-mR1,CNAG_04331,HIGH,STOP_LOST,60.0,0.83
4,VNBI,NRHc5022.ENR,var_VNBI_315876,CNAG_04337-mR1,CNAG_04337,HIGH,SPLICE_SITE_DONOR+SPLICE_SITE_REGION+INTRON,60.0,0.82
...,...,...,...,...,...,...,...,...,...
168797,VNBII,PMHc1002.ENR,var_VNBII_276666,None,CNAG_04857+CNAG_04858,HIGH,GENE_FUSION_REVERESE,NaN,NaN
168798,VNII,8-1,var_VNII_69136,None,CNAG_04739+CNAG_04740,HIGH,GENE_FUSION_REVERESE,NaN,NaN
168799,VNBI,Bt142,var_VNBI_79359,None,CNAG_03878+CNAG_07482,HIGH,GENE_FUSION_REVERESE,NaN,NaN
168800,VNI,AD6-55a,var_VNI_41401,None,CNAG_03671+CNAG_03670,HIGH,GENE_FUSION_REVERESE,NaN,NaN


Get the annotation of the reference genomes (including repeat_fraction and comparison to main reference) and join it to the strains.

In [61]:
con = duckdb.connect(database=mydb, read_only=False)
query = f"""SELECT gff.lineage,samples.strain,
            gff.gene_id, gff.gene_name, gff.description,
            gff.feature_id AS transcript_id, gff.repeat_fraction,
            gff.identical_to_main_ref, gff.start_stop_mutations
            FROM gff
            JOIN samples ON samples.lineage = gff.lineage
            WHERE gff.lineage IN {lineages} and gff.primary_tag = 'mRNA'
            """
annotation = con.execute(query).fetch_df()
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [62]:
annotation

,lineage,strain,gene_id,gene_name,description,transcript_id,repeat_fraction,identical_to_main_ref,start_stop_mutations
0,VNI,TCl-14,CNAG_06290,HXT2,Low-affinity glucose transporter HXT2,CNAG_06290-mR2,0.00,Yes,
1,VNI,TCl-14,CNAG_06291,FPD1,polysaccharide deacetylase,CNAG_06291-mR1,0.00,Yes,
2,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR1,0.00,Yes,
3,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR2,0.00,Yes,
4,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR3,0.00,Yes,
...,...,...,...,...,...,...,...,...,...
3002668,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06286,None,hypothetical protein,CNAG_06286-mR1,0.00,Yes,
3002669,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06287,GPX2,Glutathione peroxidase,CNAG_06287-mR1,0.00,Yes,
3002670,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06288,None,RNA binding protein,CNAG_06288-mR1,0.00,Yes,
3002671,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06289,None,hypothetical protein,CNAG_06289-mR1,0.04,Yes,


Join the annotation to the effects table to have all the posible gene-strain combinations  
and which of them have effects.

In [63]:
effects_annotation = annotation.merge(effects, on=['lineage', 'strain', 'gene_id','transcript_id'], how = "left")

Add status. No variant or variant.

In [64]:
effects_annotation['status'] = effects_annotation['var_id'].apply(
    lambda x: "No variant" if pd.isna(x) else "Variant")

In [65]:
effects_annotation

,lineage,strain,gene_id,gene_name,description,transcript_id,repeat_fraction,identical_to_main_ref,start_stop_mutations,var_id,impact,effect_type,mean_mapq,mean_depth_normalized,status
0,VNI,TCl-14,CNAG_06290,HXT2,Low-affinity glucose transporter HXT2,CNAG_06290-mR2,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
1,VNI,TCl-14,CNAG_06291,FPD1,polysaccharide deacetylase,CNAG_06291-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
2,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR2,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
4,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR3,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040239,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06286,None,hypothetical protein,CNAG_06286-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3040240,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06287,GPX2,Glutathione peroxidase,CNAG_06287-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3040241,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06288,None,RNA binding protein,CNAG_06288-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3040242,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06289,None,hypothetical protein,CNAG_06289-mR1,0.04,Yes,,NaN,NaN,NaN,NaN,NaN,No variant


In [66]:
effects_annotation['status'].value_counts()

status
No variant    2871522
Variant        168722
Name: count, dtype: int64

## Apply filters to table of annotation with effects
Keep only the surviving variants from the step before and the transcript quality.

### Filter by phylogenetic category

In [67]:
effects_annotation_f1 = effects_annotation.copy()
remaining_variants = effects_annotation_f1['status'] == "Variant"
effects_annotation_f1 .loc[remaining_variants, 'status'] = effects_annotation_f1.loc[remaining_variants, 'var_id'].apply(
    lambda x: "Filtered_phylogeny" if x not in vars_presence_phylo_filtered['var_id'].values else "Variant")

### Filter by percentage of strains

In [68]:
effects_annotation_f2 = effects_annotation_f1.copy()
remaining_variants = effects_annotation_f2['status'] == "Variant"
effects_annotation_f2.loc[remaining_variants, 'status'] = effects_annotation_f2.loc[remaining_variants, 'var_id'].apply(
    lambda x: "Filtered_percent_strains" if x not in vars_percent_filtered['var_id'].values else "Variant")

### Filter by start and stop mutations in reference annotation

In [69]:
effects_annotation_f3 = effects_annotation_f2.copy()
remaining_variants = effects_annotation_f3['status'] == "Variant"
if remove_mutated_genes:
    effects_annotation_f3.loc[remaining_variants, 'status'] = effects_annotation_f3.apply(
        lambda x: "Filtered_mutation" if x['start_stop_mutations'] != "" else "Variant",
        axis=1)
else:
    pass

### Filter by MAPQ of transcript

In [70]:
effects_annotation_f4 = effects_annotation_f3.copy()
remaining_variants = effects_annotation_f4['status'] == "Variant"
effects_annotation_f4.loc[remaining_variants, 'status'] = effects_annotation_f4.apply(
    lambda x: "Filtered_mapq" if x['mean_mapq'] < min_mapq else "Variant",
    axis=1)

### Filter by depth of transcript

In [71]:
effects_annotation_f5 = effects_annotation_f4.copy()
remaining_variants = effects_annotation_f5['status'] == "Variant"
effects_annotation_f5.loc[remaining_variants, 'status'] = effects_annotation_f5.apply(
    lambda x: "Filtered_depth" if (x['mean_depth_normalized'] < min_depth) | (x['mean_depth_normalized'] > max_depth) else "Variant",
    axis=1)

### Filter by fraction of repetitive sequences of the reference transcript

In [72]:
effects_annotation_f6 = effects_annotation_f5.copy()
remaining_variants = effects_annotation_f6['status'] == "Variant"
effects_annotation_f6.loc[remaining_variants, 'status'] = effects_annotation_f6.apply(
    lambda x: "Filtered_repeats" if x['repeat_fraction'] > max_repeat else "Variant",
    axis=1)

In [73]:
effects_annotation_f6['status'].value_counts()

status
No variant                  2871522
Filtered_phylogeny           135212
Variant                       13271
Filtered_percent_strains      10778
Filtered_mutation              8994
Filtered_repeats                259
Filtered_depth                  196
Filtered_mapq                    12
Name: count, dtype: int64

In [74]:
effects_annotation_f6

,lineage,strain,gene_id,gene_name,description,transcript_id,repeat_fraction,identical_to_main_ref,start_stop_mutations,var_id,impact,effect_type,mean_mapq,mean_depth_normalized,status
0,VNI,TCl-14,CNAG_06290,HXT2,Low-affinity glucose transporter HXT2,CNAG_06290-mR2,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
1,VNI,TCl-14,CNAG_06291,FPD1,polysaccharide deacetylase,CNAG_06291-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
2,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR2,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
4,VNI,TCl-14,CNAG_06292,None,"sugar transporter, variant 2",CNAG_06292-mR3,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040239,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06286,None,hypothetical protein,CNAG_06286-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3040240,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06287,GPX2,Glutathione peroxidase,CNAG_06287-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3040241,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06288,None,RNA binding protein,CNAG_06288-mR1,0.00,Yes,,NaN,NaN,NaN,NaN,NaN,No variant
3040242,VNI,NRHc5044.REL.INI.CLIN.ISO,CNAG_06289,None,hypothetical protein,CNAG_06289-mR1,0.04,Yes,,NaN,NaN,NaN,NaN,NaN,No variant


## Count the variants per gene

Remove the variant IDs of the Filtered rows.

In [ ]:
effects_annotation_variants = effects_annotation_f6.copy()
effects_annotation_variants['var_id_surviving'] = effects_annotation_variants['var_id']
effects_annotation_variants.loc[effects_annotation_variants['status'] != 'Variant', 'var_id_surviving'] = float('nan')

Total variants, filtered out, remaining variants

In [ ]:
effects_annotation_variants['var_id'].nunique(), effects_annotation_variants['var_id'].nunique() -  effects_annotation_variants['var_id_surviving'].nunique(),  effects_annotation_variants['var_id_surviving'].nunique()

(11917, 7015, 4902)

Group by gene and strain and count the number of variants.

In [78]:
effects_annotation_variants['gene_name'] = effects_annotation_variants['gene_name'].apply(lambda x: "" if pd.isna(x) else x)
gene_strain_num_vars = effects_annotation_variants.groupby(["lineage", "strain", "gene_id", "gene_name", "description"])['var_id_surviving'].nunique().reset_index(name='num_vars')
gene_strain_num_vars['presence_vars'] = gene_strain_num_vars['num_vars'].apply(lambda x: 0 if x == 0 else 1)

In [79]:
gene_strain_num_vars

,lineage,strain,gene_id,gene_name,description,num_vars,presence_vars
0,VNBI,Bt102,CNAG_00001,,hypothetical protein,0,0
1,VNBI,Bt102,CNAG_00006,,prohibitin-2,0,0
2,VNBI,Bt102,CNAG_00007,,methionine-tRNA ligase,0,0
3,VNBI,Bt102,CNAG_00008,,hypothetical protein,0,0
4,VNBI,Bt102,CNAG_00009,,calcium-binding protein,0,0
...,...,...,...,...,...,...,...
2677521,VNII,WM626,CNAG_08025,,hypothetical protein,0,0
2677522,VNII,WM626,CNAG_08026,,hypothetical protein,0,0
2677523,VNII,WM626,CNAG_08027,,hypothetical protein,0,0
2677524,VNII,WM626,CNAG_08028,,hypothetical protein,0,0


## Write the table of variants per gene

In [82]:
vars_per_gene = gene_strain_num_vars.drop(['gene_name', 'description'], axis=1)
vars_per_gene.to_csv(f'/FastData/czirion/Crypto_Desjardins/fungal_pop/data/variants_per_gene/{impact}_{lin_names}_{phylogenetic_category_name}_{below_percent_strains}_{remove_mutated_genes_name}_{min_mapq}_{min_depth}_{max_depth}_{max_repeat}.csv', index=False)

## Count the number of strains with variants in each gene

In [80]:
num_strains_per_gene = gene_strain_num_vars.groupby(["lineage", "gene_id", 'gene_name', 'description'])['presence_vars'].sum().reset_index(name='num_strains')
num_strains_per_gene.sort_values(by='num_strains', ascending=False, inplace=True)

In [81]:
num_strains_per_gene

,lineage,gene_id,gene_name,description,num_strains
6456,VNBI,CNAG_07553,EZH2,polycomb protein e(z),120
1754,VNBI,CNAG_01973,ZFC2,C2H2 zinc finger protein Zas1A,120
3442,VNBI,CNAG_03876,,Ras family protein,120
5966,VNBI,CNAG_06695,MAR1,Macrophage activating regulator of cell wall-1,119
5744,VNBI,CNAG_06438,,hypothetical protein,119
...,...,...,...,...,...
9903,VNBII,CNAG_03405,,hypothetical protein,0
9902,VNBII,CNAG_03404,,"hypothetical protein, variant",0
9901,VNBII,CNAG_03403,,mitochondrial protein,0
9900,VNBII,CNAG_03402,,hypothetical protein,0


# Get the presence of variants per strain to pass to the phylogenetic analysis

In [83]:
# con = duckdb.connect(database=mydb, read_only=False)
# query = f"""SELECT presence.var_id, samples.strain
#             FROM presence
#             JOIN strains ON presence.sample = samples.sample 
#             """
# vars_presence = con.execute(query).fetch_df()
# con.close()

In [84]:
# vars_presence.to_csv(f'/FastData/czirion/Crypto_Desjardins/fungal_pop/data/vars_presence.csv', index=False)